In [1]:
import time
import random
import collections
import numpy as np
from causality_functions import jitter, dependent, simulations_data
from causality_functions import compute_regression, compute_correlation
from causality_functions import add_confidence_stats, confidence_graphs
from causality_functions import stats_graphs
from causality_functions import check_generation

import altair as alt
import pandas as pd
import plotly.express as px
alt.data_transformers.disable_max_rows()

samplesize = 500
Am, Bm, Cm, = 40, 10, 30
Xm, Ym = 1, 1
Astd,Bstd,Cstd = 5, 13, 12
Xstd,Ystd = 1, 1

m1, k1 = 1, 0
m2, k2 = 1.5, 0
m3, k3 = 0.15, 0 # this is feedback term
m4, k4 = 0.75, 0.0

dg = 0.1
d1 = 1
d2 = 1
d3 = 1
d4 = 1

errorA = 2
errorB = 4
errorC = 3.5
errorX = 0.75
errorY = 0.6

mrange1 = 0,5 #slope ranges
mrange2 = 0,5
mrange3 = 0,5
mrange4 = 0,5

erangeA = 0,1 #error range
erangeB = 0.2,10 #error range
erangeC = 0.2,10 #error range
erangeX = 0.2,5 #error range
erangeY = 0.2,5 #error range


def linear_positive_C_to_B(n=100):

    def compute_B(history, generation):
        if check_generation(generation, d1):
             A, _, _ = history[-1 - d1]   
        else:
             A, _, _ = history[-1]
        if check_generation(generation, d3):
            _, _, C = history[-1 - d3]
        else:
            _, _, C = history[-1]
        _, B, _ = history[-1]
        return jitter(B + A*m1 + C*m3 - B*m2 -dg*B + k1 ,  errorB)
        

    def compute_C(history, generation):
        if check_generation(generation, d2):
            _, B, _ = history[-1 - d2]
        else:
            _, B, _ = history[-1]
        _, _, C = history[-1]
        return jitter(C + B*m2 - dg*C + k2, errorC)

    def next_generation(history, generation):
        A, _, _ = history[-1]
        A_ = jitter(A, errorA) # previous A + error

        # compute B
        B_ = compute_B(history, generation)
        # compute C
        C_ = compute_C(history, generation)
        
        return A_, B_, C_

    A = np.random.normal(loc=Am, scale=Astd) # normal distribution, Am - mean and Astd - standard deviation
    history = collections.deque(maxlen=max([d1, d2, d3]) + 1)
    history.append((A, 0, 0))
    for i in range(n + max([d1, d2, d3])):
        history.append(next_generation(history, i))

    return np.array(history[-1])


def linear_positive_A_to_C(n=100):    

    
    def compute_A(history, generation):
        A, _, _ = history[-1]
        return jitter(A, errorA)
        
    def compute_B(history, generation):
        if check_generation(generation, d1):
            A, _, _ = history[-1 - d1]
        else:
            A, _, _ = history[-1]
            
        return dependent(A, m1, k1 ,  errorB)
        

    def compute_C(history, generation):
        if check_generation(generation, d1):
            A, _, _ = history[-1 - d1]
        else:
            A, _, _ = history[-1]
        
        if check_generation(generation, d2):
            _, B, _ = history[-1 - d2]
        else:
            _, B, _ = history[-1]
            
        _, _, C = history[-1]
        return jitter(A*m3+ B*m2+ -dg*C + k2, errorC)
        
            
    
    def next_generation(history, generation):
        A_ = compute_A(history, generation)
        B_ = compute_B(history, generation)
        C_ = compute_C(history, generation)
        return A_, B_, C_

    A = np.random.normal(loc=Am, scale=Astd) # normal distribution, Am - mean and Astd - standard deviation
    history = collections.deque(maxlen=max([d1, d2, d3]) + 1)
    history.append((A, 0, 0))
    for i in range(n + max([d1, d2, d3])):
        history.append(next_generation(history, i))

    return np.array(history[-1])

def update_slopes():
    def select(lower, upper):
        return lower + random.random()*(upper-lower)
    
    global m1, m2, m3, m4
  
    m1 = select(*mrange1)
    m2 = select(*mrange2)
    m3 = select(*mrange3)
    m4 = select(*mrange4)

def update_errors():
    def select(lower, upper):
        return lower + random.random()*(upper-lower)
    
    global errorA, errorB, errorC, errorX, errorY

    errorA = select(*erangeA)
    errorB = select(*erangeB)
    errorC = select(*erangeC)
    errorX = select(*erangeX)
    errorY = select(*erangeY)
    
def overall_simulation(n=100):
    stats = []
    data = []
    for i in range(n):
        update_slopes()
        update_errors()
        ABC = simulations_data(linear_positive_C_to_B, samplesize) # you can change pathway here
        r = compute_regression(ABC)
        r.update(compute_correlation(ABC))
        stats.append(r)
        data.append(ABC)
    return pd.DataFrame(stats), np.array(data)

d, ABC_all = overall_simulation()

In [2]:
add_confidence_stats(d, ABC_all)
confidence_graphs(d)

alt.VConcatChart(...)

In [3]:
stats_graphs(d)

alt.VConcatChart(...)

In [4]:
A, B, C = random.choice(ABC_all).transpose()
AB = alt.Chart(pd.DataFrame({"A":A, "B":B})).mark_circle().encode(
    x="A",
    y="B")
BC= alt.Chart(pd.DataFrame({"B":B, "C":C})).mark_circle().encode(
    x="B",
    y="C")
AC = alt.Chart(pd.DataFrame({"A":A, "C":C})).mark_circle().encode(
    x="A",
    y="C")

alt.vconcat(AB, BC, AC)

alt.VConcatChart(...)

In [5]:
d

,kAB,kBC,kAC,mAB,mBC,mAC,r_sqrAB,r_sqrBC,r_sqrAC,r_E,...,rAB2*rBC2-rAC2,r_E_BA_C2-rBC2,mAB*mBC-mAC,mAB*mBC,confidence_rAC,confidence_residual_corr,confidence_corrected_bc_corr,confidence_slope_AC,L,U
0,1.047800e+24,1.772079e+14,7.745486e+23,1.709938e+22,0.739214,1.264010e+22,0.378736,1.0,0.378736,-0.107721,...,-1.214087e-10,-0.996498,-2.025964e+12,1.264010e+22,within,more,less,within,1.0,1.0
1,5.342144e+36,1.069637e+28,1.805144e+36,4.922744e+34,0.337906,1.663427e+34,0.001660,1.0,0.001660,-0.005578,...,-4.064669e-11,-0.980021,-2.036017e+26,1.663427e+34,within,within,less,within,1.0,1.0
2,3.206183e+14,-1.125000e+00,1.028794e+14,1.129232e+13,0.320878,3.623457e+12,0.006548,1.0,0.006548,0.031905,...,4.310788e-16,-0.996894,1.157227e-01,3.623457e+12,within,within,,within,NaN,NaN
3,1.173026e+13,-1.625000e+01,1.992860e+13,6.326277e+12,1.698906,1.074775e+13,0.799889,1.0,0.799889,0.031306,...,1.965095e-14,-0.998380,1.269531e-01,1.074775e+13,within,within,,within,NaN,NaN
4,1.563846e+15,-1.250000e+00,3.479333e+14,1.612629e+13,0.222486,3.587868e+12,0.000080,1.0,0.000080,0.090658,...,-3.848918e-18,-0.997013,-1.030273e-01,3.587868e+12,within,less,less,within,1.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,2.458818e+64,0.000000e+00,5.539175e+63,3.537300e+62,0.225278,7.968758e+61,0.358629,1.0,0.358629,-0.100042,...,-4.440892e-16,-0.995644,2.283596e+46,7.968758e+61,within,more,less,within,1.0,1.0
96,2.737956e+21,-1.558424e+14,3.564473e+21,2.824568e+19,1.301874,3.677232e+19,0.135361,1.0,0.135361,-0.008152,...,-4.221305e-09,-0.982820,-5.733826e+11,3.677232e+19,within,within,less,within,1.0,1.0
97,4.112597e+57,0.000000e+00,1.050760e+57,7.279538e+55,0.255498,1.859906e+55,0.409946,1.0,0.409946,-0.046005,...,3.885781e-16,-0.996209,0.000000e+00,1.859906e+55,within,within,less,within,1.0,1.0
98,1.836573e+37,9.429952e+26,6.349548e+36,1.126530e+36,0.345728,3.894730e+35,0.342221,1.0,0.342221,-0.121600,...,-3.012740e-11,-0.999393,-1.714337e+25,3.894730e+35,within,more,,within,NaN,NaN
